In [1]:
from functools import partial

import astroplan as ap
import astropy.units as u

from astropaul.database import database_connection, html_path
import astropaul.targetlistcreator as tlc
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

%load_ext autoreload
%autoreload 2


In [6]:
name = "APO Observing 2026-Q1"
html_dir = html_path() / name
html.clear_directory(html_dir)

session = tlc.ObservingSession(ap.Observer.at_site("APO"))

session.add_full_day("2026-03-01")
session.add_full_day("2026-03-02")
session.add_half_day("2026-03-03", first_half=True)
session.add_full_day("2026-03-04")
session.add_full_day("2026-03-05")

synthetic_phase_percent = 0.02

phase_event_defs = [
    ph.PhaseEventDef("Mid Eclipse", ph.calc_mid_eclipse),
    ph.PhaseEventDef("Eclipse", ph.calc_mid_eclipse),
    ph.PhaseEventDef(
        "Not in Eclipse", partial(ph.calc_time_of_gress, synthetic_phase_percent=synthetic_phase_percent, ingress=False)
    ),
    ph.PhaseEventDef("Eclipse", partial(ph.calc_time_of_gress, synthetic_phase_percent=synthetic_phase_percent, ingress=True)),
]

min_altitude = 35 * u.deg
max_magnitude = 13

with database_connection() as conn:
    creator = tlc.TargetListCreator(name=name, connection=conn, phase_event_defs=phase_event_defs)
    creator.steps = [
        tlc.add_targets,
        partial(tlc.filter_targets, criteria=lambda df: df["Target Type"].isin(["QuadEB", "SextEB"])),
        partial(tlc.filter_targets, inverse=True, criteria=lambda df: df["Target Name"].str.contains("Zasche")),
        tlc.ancillary_data_from_tess,
        partial(tlc.filter_targets, criteria=lambda df: df["Vmag"] < max_magnitude),
        partial(
            tlc.add_observability,
            observing_session=session,
            calc_moon_distance=True,
            observability_threshold=(min_altitude, 80 * u.deg),
        ),
        partial(tlc.filter_targets, criteria=lambda df: df["Observable Any Night"]),
        partial(tlc.add_database_table, table_name="Ephemerides"),
        partial(tlc.add_database_table, table_name="DSSI Observations"),
        # partial(
        #     tlc.add_system_configuration,
        #     table_name="DSSI Observations",
        #     time_column="Mid JD",
        #     eclipse_table="SIDE Observations",
        # ),
        partial(tlc.add_database_table, table_name="Speckle Detections"),
        partial(
            tlc.add_phase_events,
            observing_session=session,
            phase_event_defs=phase_event_defs,
            event_types=["Mid Eclipse", "Eclipse"],
        ),
        partial(tlc.add_database_table, table_name="PEPSI Observations"),
        # tlc.add_pepsi_evaluations,
    ]
    tl = creator.calculate(verbose=False)

# html.render_observing_pages(tl, None, {}, html_dir)
print(tl.summarize())

Name: APO Observing 2026-Q1
Criteria:
  lambda df: df["Target Type"].isin(["QuadEB", "SextEB"])
  Inverse of: lambda df: df["Target Name"].str.contains("Zasche")
  lambda df: df["Vmag"] < 13
  Observability calculated at APO in 15.0 min intervals
  AltitudeConstraint: {'min': 35.0, 'max': 80.0, 'boolean_constraint': True}
  lambda df: df["Observable Any Night"]
  
89 targets:
    84 QuadEB
     5 SextEB
Column Count (primary, secondary):
    Target: (3, 4)
    TESS Data: (2, 123)
    Observable: (5, 20)
    Count: (4, 0)
Associated tables:
       5 rows,  2 columns: Lunar Phases
     256 rows,  7 columns: Ephemerides
     168 rows,  9 columns: DSSI Observations
     144 rows, 12 columns: Speckle Detections
     867 rows, 10 columns: Phase Events
     180 rows, 14 columns: PEPSI Observations



In [16]:
html.clear_directory(html_dir)

altitude_categories = [
    ((-90, min_altitude.value), 0),
    ((min_altitude.value, 90), 1),
]

pl = pr.PriorityList(tl, session, interval=30 * u.min)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
pr.prioritize_phase_sequence(pl, ["Eclipse"], "Eclipse", True, True, True)
# pr.prioritize_side_observation(pl, side_state="Eclipse")
pr.calculate_overall_priority(pl)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.01)
pl.categorize_priorities(bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"])

html.render_observing_pages(tl, pl, {}, html_dir)
# pl.categorical_priorities[0]